# CS 440 Project on Rubik's Cube solving with Reinforcement Learning

_by David Edwards, December 12, 2017_

## Introduction

My son has long been fascinated by Rubik's Cubes, and yet seems to be completely incapable of solving them.  I decided to try out his 2x2 cube.  <img src="2x2.jpg">

I grew frustrated and failed even faster, and rather than research the existing solutions that were online, I decided to throw one of our algorithms and some brute force at it.  

When the proposal time came around, I realized that the Reinforcement Learning process was the hardest for me to understand, and yet when I understood it, I felt like I really understood it, so wanted to go with that approach.

## Methods

I foresaw three main problems with the task I had chosen.

1. Representing the state of the puzzle
2. Creating the MakeMoves function
3. Ensuring it runs in memory

Here was my approach:


I tried using some Python text graphics libraries to display the sides in their actual color, thinking that would make things easier.  I tried using [termcolor](https://pypi.python.org/pypi/termcolor) but found that the colors didn't match the actual Rubik's cube colors of Red, Blue, Yellow, Orange, White and Green.  There were existing Rubik's Cube Python libraries like [PyCuber](https://github.com/adrianliaw/PyCuber) and this fancy [interactive cube](https://jakevdp.github.io/blog/2012/11/26/3d-interactive-rubiks-cube-in-python/) but I wanted to do as much as possible from scratch.  So, I made the cube a nested list of list of lists.  The lowest level lists represented the top two colors of a side, the next the bottom two colors.  Then, the next level of lists was one whole face of the cube, and the final level was all six sides of the cube.

In [32]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
completeState

[[['Red', 'Red'], ['Red', 'Red']],
 [['Blue', 'Blue'], ['Blue', 'Blue']],
 [['Yellow', 'Yellow'], ['Yellow', 'Yellow']],
 [['Orange', 'Orange'], ['Orange', 'Orange']],
 [['White', 'White'], ['White', 'White']],
 [['Green', 'Green'], ['Green', 'Green']]]

Then, I needed to figure out how to display a cube in a state I could grasp with a little bit of confidence.  I created these functions to help with that.  The initial variables are my attempt at using something like constants in Python, which doesn't seem to really support them.  But, I wanted the logic for these to make sense to me later, so I just created these values, even though if I wasn't careful, they could be overwritten later.

One bonus was that each color used in a cube had different first letters, so I could just display the first letter, making it slightly easier to read.

In [18]:
FRONT = 1
LEFT = 0
TOP = 2
BOTTOM = 3
RIGHT = 4
BACK = 5

def printFirstLetters(pair):
    '''
    Prints the first letter of a given pair.
    :param pair: list containing two colors'''
    return("{}\t{}\t".format(pair[0][:1],pair[1][:1]))


def printState(state):
    '''
    Prints a 2x2 Rubik's cube state.  
    :param state: list of list of lists representing Rubik's cube state
    :return: prints out the state nicely.
    '''
    print("\t\t", printFirstLetters(state[TOP][0]))
    print("\t\t", printFirstLetters(state[TOP][1]))
    print(printFirstLetters(state[LEFT][0]), printFirstLetters(state[FRONT][0]),printFirstLetters(state[RIGHT][0]),printFirstLetters(state[BACK][0]))
    print(printFirstLetters(state[LEFT][1]), printFirstLetters(state[FRONT][1]),printFirstLetters(state[RIGHT][1]),printFirstLetters(state[BACK][1]))
    print("\t\t", printFirstLetters(state[BOTTOM][0]))
    print("\t\t", printFirstLetters(state[BOTTOM][1]))
    print("---------------------------------------------------------")


In [19]:
printState(completeState)

		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


This represents my "unfolding" of a cube.  In this case, the blue face was the one facing "me", the red to the left, the yellow on top, orange on the bottom, white to my right, green behind.  The line underneath is a separator, to make it easier to visualize when I'm displaying tens or hundreds of them.

Next, I had to make a validMove and makeMove function.

In [21]:
def validMoves(state):
    '''
    Returns a list of lists representing valid 2x2 rubik's cube moves from the given state.
    Move Rotations courtesy: http://www.rubiksplace.com/move-notations/
    :param state: list of lists representing tower of hanoi state
    :return: list of lists representing valid tower of hanoi moves from the given state.
    '''

    # evaluate if we need the front and back moves
    # validStates = ["U", "U'", "D", "D'", "R", "R'", "L", "L'", "F", "F'", "B", "B'"]

    validStates = ["U", "Uprime", "D", "Dprime", "R", "Rprime", "L", "Lprime"]

    return validStates


def makeMove(state, move, printMoves=False):
    '''
    Takes a move and makes it 2x2 rubik's cube
    :param state: list of lists representing cube
    :param move: possible rubik's cube move
    :return:the state after the move was made
    '''

    if move == "U":
        state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0] = state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0]
    if move == "Uprime":
        state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0] = state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0]
    if move == "D":
        state[LEFT][1], state[FRONT][1], state[RIGHT][1], state[BACK][1] = state[FRONT][1], state[RIGHT][1], state[BACK][1], state[LEFT][1]
    if move == "Dprime":
        state[FRONT][1], state[RIGHT][1], state[BACK][1], state[LEFT][1] = state[LEFT][1], state[FRONT][1], state[RIGHT][1], state[BACK][1]
    if move == "R":
        state[TOP][0][1], state[TOP][1][1], state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1] = state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1],state[TOP][0][1], state[TOP][1][1]
    if move == "Rprime":
        state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1], state[TOP][0][1], state[TOP][1][1] = state[TOP][0][1], state[TOP][1][1], state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1]
    if move == "L":
        state[TOP][0][0], state[TOP][1][0], state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0] = state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0],state[TOP][0][0], state[TOP][1][0]
    if move == "Lprime":
        state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0], state[TOP][0][0], state[TOP][1][0] = state[TOP][0][0], state[TOP][1][0], state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0]

    if printMoves:
        printState(state)

    return state

I have gone through several iterations of the validMoves function, and I'm not convinced this is the correct one.  I use the move notation found [here](http://www.rubiksplace.com/move-notations/), where U is move the top level clockwise, and Uprime moves the top level counter-clockwise.  Same with Down, Right, and Left.  Some had Back as well.  I may simplify these to reduce the size of the Q table.  Up, Up, Up is the same as UpPrime, so I want to minimize the total number of State,ValidMove pairs in the Q table, due to the size of the Rubik's Cube which has [3674160](https://www.therubikzone.com/number-of-combinations/) states.  I would rather have that be multiplied by 4 or 5 valid moves, rather than 8 or 10.

Regardless, let's make sure the makeMove works:

In [36]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
print("Complete State:")
printState(completeState)
print("Left side rotation:")
x = makeMove(completeState, "L", True)

Complete State:
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
Left side rotation:
		 B	Y	
		 B	Y	
R	R	 O	B	 W	W	 Y	G	
R	R	 O	B	 W	W	 Y	G	
		 G	O	
		 G	O	
---------------------------------------------------------


Here you can see we moved the left side of the cube, which pivots the Blue front to the top, the Orange bottom to the front, Green back to the bottom, and Yellow top to the back.  If we do this two more times, it should be back to the start state:

In [37]:
for i in range(3):
    makeMove(completeState,"L",True)


		 O	Y	
		 O	Y	
R	R	 G	B	 W	W	 B	G	
R	R	 G	B	 W	W	 B	G	
		 Y	O	
		 Y	O	
---------------------------------------------------------
		 G	Y	
		 G	Y	
R	R	 Y	B	 W	W	 O	G	
R	R	 Y	B	 W	W	 O	G	
		 B	O	
		 B	O	
---------------------------------------------------------
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


In addition, performing a move, and then it's prime should return to the original state:

Then, we need to determine if a move consists of a winner.  You'll see remnants of my first attempt, which basically checked to see if the current state was equal to the completeState above.  However, I realized that misses out on 5 other complete states, when the cube faces are all of the same color, but the colors aren't in the same position as they initially were.  I fixed that.

In [54]:
def winner(state):
    '''
    Determines if a winning state occured
    :param state: list of lists representing tower of hanoi state
    :return: True if winning state, False otherwise.
    '''
    completeState = [[["Red", "Red"], ["Red", "Red"]], [["Blue", "Blue"], ["Blue", "Blue"]],
                     [["Yellow", "Yellow"], ["Yellow", "Yellow"]], [["Orange", "Orange"], ["Orange", "Orange"]],
                     [["White", "White"], ["White", "White"]], [["Green", "Green"], ["Green", "Green"]]]

    return faceComplete(state[LEFT]) and faceComplete(state[FRONT]) and faceComplete(state[TOP]) and faceComplete(state[BOTTOM]) and faceComplete(state[RIGHT]) and faceComplete(state[BACK]) 

    # return state == completeState

def faceComplete(face):
    return (face[0][0]== face[0][1] == face[1][0] == face[1][1])

In [56]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
print(winner(completeState))
makeMove(completeState,'R',True)
print(winner(completeState))
makeMove(completeState,'Rprime',True)
print(winner(completeState))

True
		 Y	B	
		 Y	B	
R	R	 B	O	 W	W	 G	Y	
R	R	 B	O	 W	W	 G	Y	
		 O	G	
		 O	G	
---------------------------------------------------------
False
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
True


In [40]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
printState(completeState)
_ = makeMove(completeState,"R",True)
_ = makeMove(completeState,"Rprime", True)

		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
		 Y	B	
		 Y	B	
R	R	 B	O	 W	W	 G	Y	
R	R	 B	O	 W	W	 G	Y	
		 O	G	
		 O	G	
---------------------------------------------------------
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


We also need to be able to get a Tuple from the state,move pair which, given that a state is a three-deep list of lists, that was made a little more difficult.  Here's what I came up with:

In [60]:
def getTuple(state, move):
    '''
    Need immutable type for key to dictionary
    :param state: list of lists representing rubik's cube state
    :return: tuple representation of the state
    '''
    superTuple = tuple(tuple(tuple(s[0])+tuple(s[1])) for s in state)
    return (superTuple, move)

In [61]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
getTuple(completeState,'R')

((('Red', 'Red', 'Red', 'Red'),
  ('Blue', 'Blue', 'Blue', 'Blue'),
  ('Yellow', 'Yellow', 'Yellow', 'Yellow'),
  ('Orange', 'Orange', 'Orange', 'Orange'),
  ('White', 'White', 'White', 'White'),
  ('Green', 'Green', 'Green', 'Green')),
 'R')

That's not attractive, but it does appear to be a valid Tuple.  The 'R' at the very end represents a Right side turn.

Now we have the basics to start filling our Q.  I'm using the same trainQ and epsilonGreedy from [Assignment 5](https://github.com/doofusdavid/CS440-A5).  

In [62]:
def epsilonGreedy(epsilon, Q, state, validMovesF):
    '''
    Makes either a random move, or tries the move which Q indicates is the best.
    :param epsilon: A decreasing number representing the level of randomness
    :param Q: Dictionary of state,move - value pairs, with the higher values being better moves
    :param state: list of lists representing tower of hanoi state
    :param validMovesF: function returning valid moves
    :return:
    '''
    goodMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return tuple(random.choice(goodMoves))
    else:
        # Greedy Move
        Qs = np.array([Q.get(getTuple(state,m), 0.0) for m in goodMoves])
        return tuple(goodMoves[np.argmax(Qs)])


def trainQ(startState, nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    '''
    Creates and fills a dictionary, Q, representing the (state,move) - value pairs which, if followed
    should create the shortest path to the solution.
    :param nRepetitions: how many times to iterate through.  Higher numbers would generate more accurate results
    :param learningRate: How much to adjust the value part of the dictionary
    :param epsilonDecayFactor: how quickly to reduce the random factor.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: the dictionary, Q, and a list containing the number of steps it took per iteration to find the goal state
    '''
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1.0
    Q = {}
    stepList = []
    # show the moves while debuggin
    showMoves = False

    for nGames in range(maxGames):
        # reduce the randomness every pass
        epsilon *= epsilonDecayRate
        step = 0
        # hardcoded start state
        state = startState
        done = False

        while not done:
            if step % 5 == 0: print(".", end="")
            if step % 50 == 0: print("")
            step += 1
            # grab either a random or best of the known moves
            move = epsilonGreedy(epsilon, Q, state, validMovesF)

            # we don't want to change state directly, and because state is a list of lists, need to do a
            # deepcopy on it, then make the move
            stateNew = copy.deepcopy(state)
            makeMoveF(stateNew, move)

            # if we haven't encountered this state,move combo, add it to Q
            if getTuple(state, move) not in Q:
                Q[getTuple(state, move)] = 0.0  # Initial Q value for new state, move

            # print if debugging
            if showMoves:
                printState(stateNew)
            if winner(stateNew):
                # We won!  backfill Q
                # print('End State, we won!')
                Q[getTuple(state, move)] = -1.0
                done = True
                # we're keeping a list of the number of steps it took for each winning solution, so add it here.
                stepList.append(step)

            # update the Q which led us here using the learning factor, and the difference between the current state
            # and the old state
            if step > 1:
                Q[getTuple(stateOld, moveOld)] += rho * (-1 + Q[getTuple(state, move)] - Q[getTuple(stateOld, moveOld)])
                #print("Q[",getTuple(stateOld, moveOld),"]: ",Q[getTuple(stateOld, moveOld)])
            # Store the current state, move so we can access it for the next Q update
            stateOld, moveOld = state, move
            state = stateNew

    return Q, stepList

## Results

## Conclusions

In [57]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Edwards-Project.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Edwards-Project.ipynb is 716
